In [1]:
import os
import json
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
image_files = []
for r,d, f in os.walk("/home/local/ZOHOCORP/shiva-8700/forms/datasets/pdfimg/pdfimg/"):
    for i in f:
        if i[0] != "." and "checkpoint" not in i and "h" not in i:
            image_files.append("/home/local/ZOHOCORP/shiva-8700/forms/datasets/pdfimg/pdfimg/" + i)
len(image_files)

135

In [3]:
paths = {}
for name in image_files:
    file_name = name.split("/")[-1]
    paths[name] = {}
    paths[name]["orginal_labels"] = "/home/local/ZOHOCORP/shiva-8700/forms/datasets/pdfimg-labels/" + file_name + "_labels.json"
    paths[name]["features"] = "/home/local/ZOHOCORP/shiva-8700/forms/api/updated/src/jsonfiles/"   + file_name[:-4] + ".json"

In [4]:
def intersection_over_union(box_A, box_B):
    """ find the intersection over union of two bounding box """
     
    assert box_A[0] < box_A[2]
    assert box_A[1] < box_A[3]
    assert box_B[0] < box_B[2]
    assert box_B[1] < box_B[3]

    # determine the coordinates of the intersection rectangle
    x_left = max(box_A[0], box_B[0])
    y_top = max(box_A[1], box_B[1])
    x_right = min(box_A[2], box_B[2])
    y_bottom = min(box_A[3], box_B[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (box_A[2] - box_A[0]) * (box_A[3] - box_A[1])
    bb2_area = (box_B[2] - box_B[0]) * (box_B[3] - box_B[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

    pass

def match_ids(file_name):
    """ given different json files match all ids add new ids and add links """
    pass

In [5]:
# after saved data do the necessary statistical data

In [6]:
for tmp in range(len(image_files)):
    file = image_files[tmp]
    gt_name = paths[file]["orginal_labels"]
    feature_name = paths[file]["features"]
    fn = feature_name.split("/")[-1]
    try:
        with open(gt_name, "r") as f:
            gt = json.load(f)
        with open(feature_name, "r") as f:
            feature = json.load(f)
    except:
        continue

    ## link labels id -> bbox
    take_features = {}
    ## add all form field add all neigbors
    for key in feature:
        take_features[key] = feature[key]["features"]["bbox"]
        for key_1 in feature[key]["neighbors"]:
            if key_1 not in take_features:
                take_features[key_1] = feature[key]["neighbors"][key_1]["bbox"]

    id_match = {}
    for key in gt["ele"]:
        for key_1 in take_features:
            if intersection_over_union(gt["ele"][key][:4], take_features[key_1]) > 0.75:
                id_match[key] = key_1
                break

    #print(len(take_features), len(gt["ele"]))
    ## add links to features:
    feature["links"] = []
    for link in gt["links"]:
        if link[0]  in id_match and link[1]  in id_match:
            feature["links"].append([id_match[link[0]], id_match[link[1]]])

    #print(feature)
    
    with open("./jsonfiles/"+fn, "w") as f:
        json.dump(feature, f)
    






In [7]:
len(feature["links"]), len(gt["links"])

(17, 18)

In [77]:
c = 0
for l in gt["links"]:
    if l[0] not in gt["ele"] or l[1] not in gt["ele"]:
        c += 1
c

0

In [78]:
## display to check annotations

In [9]:
files = []
for _,_,f in os.walk("./jsonfiles/"):
    for i in f:
        files.append("./jsonfiles/" + i)
len(files)

134

In [17]:
box_data = []
## display
color = [(122,0,122), (0,255,0), (255,0,0), (0,0,122)]
tmp = random.randint(0, len(files) - 1)
for tmp in range(len(files)):
    file = files[tmp]
    fn = file.split("/")[-1]
    imgpath = "/home/local/ZOHOCORP/shiva-8700/forms/datasets/pdfimg/pdfimg/" + fn[:-5] + ".jpg"
    #print(imgpath)
    img = cv2.imread(imgpath)
    #print(img.shape)
    with open(file, "r") as f:
        data = json.load(f)
    ## plot every bbox based on class color
    for key in data:
        if key != "links":
            bbox = data[key]["features"]["bbox"]
            v = data[key]["features"]["class"]
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color[v], 6)


    data_all = []
    for key in data:
        if key != "links":
            for key1 in data[key]["neighbors"]:   
                if data[key]["features"]["class"] == 0:
                    data_all.append([key, key1, 0])
                elif data[key]["features"]["class"] == 1:
                    data_all.append([key, key1, 1])
                elif data[key]["features"]["class"] == 2:
                    data_all.append([key, key1, 2])


    ## display links
    for l in data["links"]:
        bbox1 = data[l[0]]["features"]["bbox"]
        bbox2 = data[l[1]]["features"]["bbox"]
        c1 = ((bbox1[0] + bbox1[2])//2, (bbox1[1] + bbox1[3])//2)
        c2 = ((bbox2[0] + bbox2[2])//2, (bbox2[1] + bbox2[3])//2)
        cv2.line(img, (c1[0], c1[1]), (c2[0], c2[1]), (255, 0, 0), 6)

    


    for d in data_all:
        if d[2] == 2:
            data_temp = data[d[0]]["neighbors"][d[1]]
            box1 = data[d[0]]["features"]["bbox"]
            box2 = data[d[0]]["neighbors"][d[1]]["bbox"]
            data_temp_features = data_temp["distance"] + data_temp["distance_score"] + data_temp["angle"] + data_temp["xintersection"] + data_temp["yintersection"]
            if [d[0], d[1]] in data["links"]:
                label = [1]
            else:
                label = [0]
            #print(data_temp_features, label)
            box_data.append(box1 + box2 + data_temp_features + label)

    
            

    ## get entire dict classify it as form -> text seperate via class and save 


#plt.figure(figsize=(10, 10))
#plt.imshow(img)
#plt.show()


In [11]:
len(all_neighbor_pairs), len(data["links"])

NameError: name 'all_neighbor_pairs' is not defined

In [132]:
data["0"]["neighbors"]["70"]

{'distance': 7.002857560004646,
 'distance_score': 0.7330030858218309,
 'angle': 179.2988942984248,
 'xintersection': 0.014056224899598393,
 'yintersection': 1.0,
 'bbox': [275, 2053, 773, 2131],
 'id': 70,
 'class': 3}

In [18]:
box_data_all = np.array(box_data)
box_data_all.shape

(8130, 142)

In [19]:
np.save("line_plus.npy", box_data_all)